In [46]:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import pandas as pd
import numpy
import torch
from transformers import BartForSequenceClassification, BartTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [53]:
def return_high_conf(data):
    LABELS = ["contradiction", "neutral", "entailment"]
    high_conf_list = []

    high_conf_dict = {
        "97-100": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
        "95-97": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
        "90-95": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
        "80-90": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
        "60-80": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
        "40-60": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
        "0-40": {"This text is about politics.": 0,
                   "This text expresses gratitude.": 0,
                   "This text expresses frustration.": 0,
                   "This text is focused on solutions.": 0,
                   "This text provides information or news.": 0,
                   "This text expresses fear or panic.": 0,
                   "This text contains blaming.": 0,
                   "This text is seeking help or advice.": 0,
                   "This text is about wildfires.": 0,
                   "This text is about prescribed burns.": 0,
                   "This text is about fire management.": 0},
    }
    
    for _, row in data.iterrows():
        premise = row["premise"]
        hypothesis = row["hypothesis"]
        label = row["label"]
        
        inputs = tokenizer(
            premise,
            hypothesis,
            return_tensors="pt",
            truncation=True,
            padding=True
        )

        scores = torch.softmax(model(**inputs).logits, dim=-1)
        scores = scores.squeeze().tolist()

        if scores[2] > .97:
            high_conf_dict["97-100"][hypothesis] += 1
        elif scores[2] >= .95 and scores[2] < .97:
            high_conf_dict["95-97"][hypothesis] += 1
        elif scores[2] >= .90 and scores[2] < .95:
            high_conf_dict["90-95"][hypothesis] += 1
        elif scores[2] >= .80 and scores[2] < .90:
            high_conf_dict["80-90"][hypothesis] += 1
        elif scores[2] >= .60 and scores[2] < .80:
            high_conf_dict["60-80"][hypothesis] += 1
        elif scores[2] >= .40 and scores[2] < .60:
            high_conf_dict["40-60"][hypothesis] += 1
        elif scores[2] < .40:
            high_conf_dict["0-40"][hypothesis] += 1
    
    return high_conf_dict

def change_hyp_phrase(data, old_hyp, new_hyp):

    data.loc[data["hypothesis"] == old_hyp, "hypothesis"] = new_hyp

    return data 

In [54]:
test_data_entail_1 = pd.read_csv("entailment_test_data_sixEight.csv")
test_data_entail_2 = pd.read_csv("entailment_test_data.csv")

In [55]:
full_entail_test = pd.concat([test_data_entail_1, test_data_entail_2], ignore_index=True)
full_entail_test

,premise,hypothesis,label
0,"Ive been fired, im Speaking out",This text expresses frustration.,entailment
1,Hey Sam. I’m also a wildland firefighter with ...,This text is focused on solutions.,entailment
2,Hey Sam. I’m also a wildland firefighter with ...,This text provides information or news.,entailment
3,If you’re feeling brave and upset and want to ...,This text provides information or news.,entailment
4,"Hey Sam, I’m a national parks reporter who cov...",This text provides information or news.,entailment
...,...,...,...
239,Maybe rig a hinging home filter for all of the...,This text is focused on solutions.,entailment
240,"I don’t have chickens (wish I did😉), but yeste...",This text provides information or news.,entailment
241,I had a mister on for a while to weigh down so...,This text is focused on solutions.,entailment
242,Thank you!!!,This text expresses gratitude.,entailment


In [58]:
full_entail_test_1 = change_hyp_phrase(full_entail_test, old_hyp="This text is political in nature.", new_hyp="This text is about politics.")
full_entail_test_1 = change_hyp_phrase(full_entail_test_1, old_hyp="This text is about fire management", new_hyp="This text is about fire management.")

high_conf_entail = return_high_conf(full_entail_test_1)

In [59]:
model_name = 'facebook/bart-large-mnli'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name, num_labels=3, multi_label=True)

/Users/joeycartwright/miniconda3/envs/csci477/lib/python3.11/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [60]:
high_conf_entail

{'97-100': {'This text is about politics.': 0,
  'This text expresses gratitude.': 6,
  'This text expresses frustration.': 1,
  'This text is focused on solutions.': 0,
  'This text provides information or news.': 1,
  'This text expresses fear or panic.': 0,
  'This text contains blaming.': 2,
  'This text is seeking help or advice.': 0,
  'This text is about wildfires.': 0,
  'This text is about prescribed burns.': 0,
  'This text is about fire management.': 0},
 '95-97': {'This text is about politics.': 0,
  'This text expresses gratitude.': 2,
  'This text expresses frustration.': 5,
  'This text is focused on solutions.': 0,
  'This text provides information or news.': 3,
  'This text expresses fear or panic.': 0,
  'This text contains blaming.': 0,
  'This text is seeking help or advice.': 0,
  'This text is about wildfires.': 0,
  'This text is about prescribed burns.': 1,
  'This text is about fire management.': 0},
 '90-95': {'This text is about politics.': 0,
  'This text ex

In [75]:
def eval_perc(data, thresh):
    perc_dict = {
      'This text is about politics.': [0,0],
      'This text expresses gratitude.': [0,0],
      'This text expresses frustration.': [0,0],
      'This text is focused on solutions.': [0,0],
      'This text provides information or news.': [0,0],
      'This text expresses fear or panic.': [0,0],
      'This text contains blaming.': [0,0],
      'This text is seeking help or advice.': [0,0],
      'This text is about wildfires.': [0,0],
      'This text is about prescribed burns.': [0,0],
      'This text is about fire management.': [0,0]
    }

    date_range_conv = {
        "97-100": 97,
        "95-97": 95,
        "90-95": 90,
        "80-90": 80,
        "60-80": 60,
        "40-60": 40,
        "0-40": 40
    }
    
    for date_range in data.keys():
        for text in data[date_range].keys():
            if date_range_conv[date_range] >= thresh:
                perc_dict[text][0] += data[date_range][text]

            perc_dict[text][1] += data[date_range][text]

    return perc_dict

def turn_into_perc(data_dict):

    perc_dict = {
      'This text is about politics.': 0,
      'This text expresses gratitude.': 0,
      'This text expresses frustration.': 0,
      'This text is focused on solutions.': 0,
      'This text provides information or news.': 0,
      'This text expresses fear or panic.': 0,
      'This text contains blaming.': 0,
      'This text is seeking help or advice.': 0,
      'This text is about wildfires.': 0,
      'This text is about prescribed burns.': 0,
      'This text is about fire management.': 0
    }

    for text in data_dict.keys():
        if data_dict[text][0] == 0:
            perc_dict[text] = 0
        else:
            perc_dict[text] = data_dict[text][0] / data_dict[text][1]

    return perc_dict

In [81]:
perc_dict_entail = eval_perc(high_conf_entail, 80)

In [82]:
turn_into_perc(perc_dict_entail)

{'This text is about politics.': 0,
 'This text expresses gratitude.': 1.0,
 'This text expresses frustration.': 0.3275862068965517,
 'This text is focused on solutions.': 0,
 'This text provides information or news.': 0.7333333333333333,
 'This text expresses fear or panic.': 0.2857142857142857,
 'This text contains blaming.': 0.24,
 'This text is seeking help or advice.': 0.2,
 'This text is about wildfires.': 0.3125,
 'This text is about prescribed burns.': 1.0,
 'This text is about fire management.': 0.13043478260869565}